# Data Acquisition and Aggregation

This notebook is to be used to show the process taken in extracting and aggregating the desired data from Excel spreadsheets contained in the `data_files` folder and written to the `aggregated_data` folder.

## Imports and Constants

In [1]:
import os            # Access individual files inside of directories.
import openpyxl      # Enables ability to read Excel Spreadsheets.
import xlrd          # Enables ability to read specific Excel Spreadsheets.
import pandas as pd  # Creates dataframes to store data and write to csv files.

DIRECTORY = 'data_files'  # Directory where spreadsheets are located.

## Defined Functions

In [2]:
def getData(filename, sheetname, headercols, dfcols):
    """
    Accesses the file and determines the engine to use to parse data.
    Uses the name of the sheet and name of the header to access data.
    Then creates and returns a dataframe with desired column names.
    """
    # Logic to get data from different excel files type.
    if filename.endswith('.xls'):
        data = pd.ExcelFile(filename, engine='xlrd')
    else:
        data = pd.ExcelFile(filename, engine='openpyxl')
    
    # Parse the spreadsheet, skipping the row that has only the state.
    data_s = data.parse(sheetname, header=[0, 1], skiprows=[2])
    data_s.head()
    
    # This into the headers and then the columns.
    data_sub = data_s[headercols]
    # This sets the names of the columns without the headers.
    data_sub.columns = dfcols
    
    return data_sub

def toColInt(df, cols):
    """
    Takes in a Pandas dataframe and a list of column names.
    Then returns a dataframe with the columns converted to integers and
    NaN values changed to 0.
    """
    # For loop to iterate through the column names and convert to integers.
    for c in cols:
        df[c] = df[c].fillna(0).astype('int')
        
    return df

def toColFloat(df, cols):
    """
    Takes in a Pandas dataframe and a list of column names.
    Then returns a dataframe with the column converted to floats and 
    NaN values changed to 0.
    """
    # For loop to iterate through the column names and convert to floats.
    for c in cols:
        df[c] = df[c].fillna(0).astype('float')
        
    return df

def parseData(filename, sheetname, headercols, dfcols, intcols=[], fcols=[]):
    """
    Invokes the getData, toColInt, and toColFloat to get data from desired spreadsheet
    and convert the columns to appropriate data types. Then returns a Pandas dataframe.
    """
    df = getData(filename, sheetname, headercols, dfcols)
    df = toColInt(df, intcols)
    df = toColFloat(df,fcols)
    
    return df


def combineData(directory, sheetname, headercols, dfcols, start_year, end_year, intcols = [], fcols = []):
    """
    Parses through multiple desired spreadsheets then extracts and aggregates the data into a single 
    Pandas dataframe.
    """
    # Initialize list to store dataframes.
    dataframes = []
    
    # For loop to iterate through directory.
    for file in os.listdir(directory):
        # Skip non-excel files.
        if not file.endswith('xls') and not file.endswith('.xlsx'):
            continue
        # Get file path.
        filepath = os.path.join(directory, file)
        # Extract the year from the file.
        year = int(file.split()[0])
        # Logic for data range.
        if start_year <= year <= end_year:
            # Create dataframe.
            df = parseData(filepath, sheetname, headercols, dfcols, intcols, fcols)
            # Get a list of columns to then insert year into the 3 position.
            cols = list(df.columns)
            cols.insert(2, 'Year')
            df = df.reindex(columns = cols)
            # Add value to Year.
            df['Year'] = year
            # Add dataframe to list.
            dataframes.append(df)
        
            # Combine all the dataframes into one.
            combined_df = pd.concat(dataframes, ignore_index = True)
        
    return combined_df

## Extract Data from Excel Spreadsheets

### Outcomes and Factor Rankings

In [3]:
# Initialize desired sheetname, data to extract, year range, and name of output csv file name.
sheetName = 'Outcomes & Factors Rankings'
headerCols = [
        ('Unnamed: 0_level_0', 'FIPS'),
        ('Unnamed: 2_level_0', 'County'),
        ('Health Outcomes', 'Rank')
    ]
dfCols = ['FIPS', 'County', 'Rank']
startYear = 2015
endYear = 2023
outputCSV = f'aggregated_data/Washington_Health_Outcome_Rankings_{startYear}-{endYear}.csv'

# Create dataframe with aggregated data.
rankings = combineData(DIRECTORY, sheetName, headerCols, dfCols, startYear, endYear)

# Write dataframe to csv.
rankings.to_csv(outputCSV, index = False)

### Ranked Measure Data

In [4]:
# Initialize desired sheetname, data to extract, year range, and name of output csv file name.
sheetName = 'Ranked Measure Data'
headerCols = [
        ('Unnamed: 0_level_0', 'FIPS'),
        ('Unnamed: 2_level_0', 'County'),
        ('Poor physical health days', 'Physically Unhealthy Days'),
        ('Poor mental health days', 'Mentally Unhealthy Days'),
        ('Adult smoking', '% Smokers'),
        ('Adult obesity', '% Obese'),
        ('Physical inactivity', '% Physically Inactive'),
        ('Access to exercise opportunities', '% With Access'),
        ('Excessive drinking', '% Excessive Drinking'),
        ('Teen births', 'Teen Birth Rate'),
        ('Uninsured', '% Uninsured'),
        ('Preventable hospital stays', 'Preventable Hospitalization Rate'),
        ('Some college', '% Some College'),
        ('Unemployment', '% Unemployed'),
        ('Children in poverty', '% Children in Poverty'),
        ('Income inequality', '80th Percentile Income'),
        ('Income inequality', '20th Percentile Income'),
        ('Income inequality', 'Income Ratio'),
        ('Children in single-parent households', '% Single-Parent Households'),
        ('Injury deaths', 'Injury Death Rate')
    ]
dfCols = ['FIPS', 'County', 'Physically Unhealthy Days', 'Mentally Unhealthy Days',
         '% Smokers', '% Obese', '% Physically Inactive', '% Exercise Access',
         '% Excessive Drinking','Teen Birth Rate', '% Uninsured', 'Preventable Hospitalization Rate', 
          '% Some College', '% Unemployed', '% Children in Poverty', '80th Percentile Income',
          '20th Percentile Income', 'Income Ratio', '% Single-Parent Households', 'Injury Death Rate',
         ]
startYear = 2015
endYear = 2023
outputCSV = f'aggregated_data/Washington_Ranked_Measure_Data_{startYear}-{endYear}.csv'

intCols = ['FIPS', '80th Percentile Income', '20th Percentile Income']
fCols = ['Physically Unhealthy Days', 'Mentally Unhealthy Days', '% Smokers', '% Obese',
         '% Physically Inactive', '% Exercise Access', '% Excessive Drinking', 'Teen Birth Rate', 
         '% Uninsured', 'Preventable Hospitalization Rate', '% Some College', '% Unemployed', 
         '% Children in Poverty','Income Ratio', '% Single-Parent Households', 'Injury Death Rate']
 

# Create dataframe with aggregated data.
measures = combineData(DIRECTORY, sheetName, headerCols, dfCols, startYear, endYear, intCols, fCols)

# Write dataframe to csv.
measures.to_csv(outputCSV, index = False)

## Population

In [5]:
# Old format
sheetName = 'Additional Measure Data'
startYear = 2015
endYear = 2020

headerCols = [
    ('Unnamed: 0_level_0', 'FIPS'),
    ('Unnamed: 2_level_0', 'County'),
    ('Demographics', 'Population')    
]
dfCols = ['FIPS', 'County', 'Population']

intCols = ['FIPS', 'Population']
fCols = []

oldPopCSV = f'aggregated_data/Washington_Population_Data_{startYear}-{endYear}.csv'

# Create Dataframe with aggregated data.
oldPop = combineData(DIRECTORY, sheetName, headerCols, dfCols, startYear, endYear, intCols, fCols)

# Write dataframe to csv.
oldPop.to_csv(oldPopCSV, index = False)

In [6]:
# Get population new format
sheetName = 'Additional Measure Data'
startYear = 2021
endYear = 2023

headerCols = [
    ('Unnamed: 0_level_0', 'FIPS'),
    ('Unnamed: 2_level_0', 'County'),
    ('Population', 'Population')
] 
dfCols = ['FIPS', 'County', 'Population']
intCols = ['FIPS', 'Population']
fCols = []

newPopCSV = f'aggregated_data/Washington_Population_Data_{startYear}-{endYear}.csv'

# Create Dataframe with aggregated data.
newPop = combineData(DIRECTORY, sheetName, headerCols, dfCols, startYear, endYear, intCols, fCols)

# Write dataframe to csv.
newPop.to_csv(newPopCSV, index = False)

In [7]:
# Combine the csv files.
combinedPopCSV = 'aggregated_data/Washington_Population_Data_2015-2023.csv'
oldDF = pd.read_csv(oldPopCSV)
newDF = pd.read_csv(newPopCSV)

allDF = pd.concat([oldDF, newDF], ignore_index = True)

allDF.to_csv(combinedPopCSV, index = False)